# Лабораторная работа № 5.
Нахождение собственных чисел и собственных векторов матрицы  
Вариант 9

In [ ]:
import numpy as np



np.set_printoptions(2)

$a_{ij}=e^{-(i+j)\alpha}$  
$i,j=1,...,n$  
$0\le \alpha\le max(i, j)$



In [ ]:
def make_matrix(n):
    low = 10 ** -6
    rng = np.random.default_rng()
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            high = 1 / max(i + 1, j + 1)
            alpha = rng.uniform(low, high)
            A[j, i] = A[i, j] = np.exp(-(i + j + 2) * alpha)

    return A

Метод скалярных произведений (частичная проблема поиска собственных значений)

In [49]:
def scalar_product_method(A, eps=1e-10, max_iterations=10**6):
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]

    n = A.shape[0]
    iterations = 0

    y_prev = np.random.rand(n)
    y1_prev = y_prev.copy()

    y_prev /= np.linalg.norm(y_prev)
    y1_prev /= np.linalg.norm(y1_prev)

    eigen_value = np.dot(y_prev, y1_prev) / np.dot(y_prev, y1_prev)
    prev_eigen_value = eigen_value

    while iterations < max_iterations:
        iterations += 1

        y_new = A @ y_prev
        y1_new = A.T @ y1_prev

        eigen_value = np.dot(y_new, y1_new) / np.dot(y_prev, y1_new)

        if abs(eigen_value - prev_eigen_value) < eps:
            break

        prev_eigen_value = eigen_value
        y_prev, y1_prev = y_new, y1_new

    eigen_vector = y_new / np.linalg.norm(y_new)

    return eigen_value, eigen_vector


Степенной метод (частичная проблема поиска собственных значений)  
Предполагается, что матрица А действительная с положительными коэффициентами

In [ ]:
def power_method(A, eps=1e-10, max_iterations=10**6):
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]

    iterations = 0
    n = A.shape[0]

    y_prev = np.random.rand(n)
    y_prev /= np.linalg.norm(y_prev)
    y_new = y_prev.copy()

    while (iterations < max_iterations):
        iterations += 1

        y_new = A @ y_prev
        y_new /= np.linalg.norm(y_new)

        if np.linalg.norm(y_new - y_prev) < eps:
            break

        y_prev = y_new

    max_eigen_value = np.dot(A @ y_new, y_new)
    eigen_vector = y_new / np.linalg.norm(y_new)

    return max_eigen_value, eigen_vector


Метод Якоби, метод вращений (полная проблема поиска собственных значений)  
Предполагается, что матрица А вещественная и симметричная

In [55]:
def measure(A):
    s = 0
    n = A.shape[0]

    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            s += A[i, j] ** 2

    return s


def max_pos(A):
    n = A.shape[0]
    m = np.finfo("float64").min
    pos = (0, 0)

    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if A[i, j] > m:
                m = A[i, j]
                pos = (i, j)

    return pos


def jacobi_method(A, eps=10**-10, max_iterations=10**6):
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]

    iterations = 0
    n = A.shape[0]
    Q = np.eye(n)

    while (measure(A) >= eps) and (iterations <= max_iterations):
        i, j = max_pos(A)

        if A[i, i] != A[j, j]:
            angle = np.arctan(2 * A[i, j] / (A[i, i] - A[j, j])) / 2
        else:
            angle = np.pi / 4

        P = np.eye(n)
        P[i, i] = P[j, j] = np.cos(angle)
        P[i, j] = -np.sin(angle)
        P[j, i] = np.sin(angle)

        A = P.T @ A @ P
        Q = Q @ P

        iterations += 1

    eigen_values = np.diag(A)
    eigen_vectors = [Q[0:n, i] for i in range(n)]

    return (eigen_values, eigen_vectors)



In [ ]:
n = 5
A = make_matrix(n)
print(f"Матрица имеет вид:\n{A}")

Матрица имеет вид:
[[0.18 0.47 0.3  0.5  0.9 ]
 [0.47 0.15 0.28 0.52 0.63]
 [0.3  0.28 0.35 0.19 0.81]
 [0.5  0.52 0.19 0.68 0.48]
 [0.9  0.63 0.81 0.48 0.19]]


In [50]:
eigen_value_s, eigen_vector_s = scalar_product_method(A)
print(f"Максимальное по модулю собс. значение: {eigen_value_s}")
print(f"Собственный вектор: {eigen_vector_s}")
print(f"Вектор невязки: {A @ eigen_vector_s - eigen_value_s * eigen_vector_s}")

Максимальное по модулю собс. значение: 2.3897920556566277
Собственный вектор: [0.46 0.4  0.38 0.45 0.54]
Вектор невязки: [-9.41e-05 -2.96e-05 -6.72e-05  3.13e-06  1.47e-04]


In [51]:
eigen_value_p, eigen_vector_p = power_method(A)
print(f"Максимальное по модулю собс. значение: {eigen_value_p}")
print(f"Собственный вектор: {eigen_vector_p}")
print(f"Вектор невязки: {A @ eigen_vector_p - eigen_value_p * eigen_vector_p}")

Максимальное по модулю собс. значение: 2.389792026090044
Собственный вектор: [0.46 0.4  0.38 0.45 0.54]
Вектор невязки: [ 1.69e-11  5.32e-12  1.20e-11 -5.18e-13 -2.63e-11]


In [53]:
B = A - eigen_value_s * np.eye(n)
b_eigen_value, b_eigen_vector = scalar_product_method(B)
min_eigen_value = b_eigen_value + eigen_value_s
print(f"Минимальное собственное значение матрицы: {min_eigen_value}")

Минимальное собственное значение матрицы: -0.8826620127516431


In [56]:
eigen_values, eigen_vectors = jacobi_method(A)
print("Собственные значения, вектора и вектора невязки:")
for i, v in enumerate(eigen_values):
    vec = eigen_vectors[i]
    print(f"{i}\t{v}")
    print(f"\t{vec}")
    print(f"\t{A @ vec - v * vec}")

Собственные значения, вектора и вектора невязки:
0	-0.7327025276815455
	[ 0.73 -0.01  0.03 -0.08 -0.67]
	[ 0.02 -0.12 -0.31 -0.07  0.02]
1	-0.27329800598047016
	[-0.17  0.91 -0.16 -0.29 -0.17]
	[ 6.00e-03 -8.64e-05 -8.86e-02 -6.95e-04  7.42e-02]
2	-0.1552480539852873
	[-0.38  0.    0.84  0.   -0.38]
	[-2.13e-01 -1.83e-01 -5.55e-17 -2.13e-01  2.11e-01]
3	0.33243035859323417
	[-0.24  0.1  -0.27  0.85 -0.37]
	[ 0.09 -0.   -0.19  0.    0.09]
4	2.3690306475163045
	[0.48 0.4  0.43 0.44 0.48]
	[-0.08  0.   -0.15  0.02  0.19]
